## 04/02 ~ continuing work with py files

In [1]:
from src.data_cleaning import data_labeling
from src.data_cleaning import feature_engineering
from src.language_processing import to_tfidf

In [2]:
import pandas as pd
#pd.set_option('display.max_columns', 100)

In [3]:
df = data_labeling('data/data_prepping_zeros_index.xlsx',
              'data/smalleyeball_partone.xlsx',
              'data/smalleyeball_parttwo.xlsx')

/home/daniel/Desktop/Seattle_g89/spam_filter/look2social_spamfilter/src/data_cleaning.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df['spam_corporate'][df.screen_name.isin(combined_lst)] = 1
/home/daniel/Desktop/Seattle_g89/spam_filter/look2social_spamfilter/src/data_cleaning.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df['spam_bot'][df.expanded_url == 'https://acrobat.adobe.com'] = 1
/home/daniel/Desktop/Seattle_g89/spam_filter/look2social_spamfilter/src/data_cleaning.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cavea

In [4]:
df2 = feature_engineering(df)

/home/daniel/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)
/home/daniel/Desktop/Seattle_g89/spam_filter/look2social_spamfilter/src/data_cleaning.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df3['time_created'] = pd.to_datetime(df3['created_at']).copy()
/home/daniel/Desktop/Seattle_g89/spam_filter/look2social_spamfilter/src/data_cleaning.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in t

In [5]:
df2.head()

,text,spam,spam_marketing,spam_hijack,spam_corporate,spam_bot,spam_known,spam_own,Docusign,onespan,...,adobe sign,listed_count,description,statuses_count,followers_count,favourites_count,friends_count,time_float_sin,time_float_cos,is_description_none
0,just closed a deal in 27 hours using #AdobeSig...,0,0,0,0,1,0,0,False,False,...,True,1,There's no substitute for experience! Call 609...,6750,108,1,12,-0.049140,0.998792,False
1,just closed a deal in 2 days using #AdobeSign ...,0,0,0,0,1,0,0,False,False,...,True,14,We provide an affordable way to have a wonderf...,5018,47,2,25,-0.501007,0.865443,False
2,just closed a deal in 2 hours using #AdobeSign...,0,0,0,0,1,0,0,False,False,...,True,0,,29,60,10,52,-0.599548,0.800339,True
3,just closed a deal in 26 hours using #AdobeSig...,0,0,0,0,1,0,0,False,False,...,True,2,,5332,66,0,182,-0.325843,0.945424,True
4,just closed a deal in 6 days using #AdobeSign ...,0,0,0,0,1,0,0,False,False,...,True,8,Carefully Crafted & Intentionally Inspired Eve...,1337,225,19,348,-0.098162,0.995170,False


In [6]:
df = df2

In [7]:
text_tfidf_matrix = to_tfidf(df,'text',20)

In [8]:
text_tfidf_matrix[0].shape

(1, 20)

In [9]:
desc_tfidf_matrix = to_tfidf(df,'description',20)

In [10]:
text_tfidf_matrix.mean()

0.0018319504148755373

In [11]:
desc_tfidf_matrix.mean()

0.00888483454963511

# Finally starting modeling!

## Approaching way

1. numerical predictors will be modeled with logistic regression, random forest, gradient boosting

2. TFIDF matrix predictors will be modeled by Naive Bayes. Currently, as I'm testing with local computer, I reduced down the number of max features to 1000. However,if I start cloud computing (Amazon AWS), I will increase the number of max features

3. Ultimately, I will ensemble three different models - one based on numerical predictors and two based on post text TFIDF matrix and description text TFIDF matrix. After evaluating all model, I will find a best ratio for the weight of each model to be ensembled.

## Spam_bot

### Logistic Regression

In [12]:
%matplotlib inline
import matplotlib.pyplot as plt
from scipy import stats
import scipy
import numpy as np
from sklearn.linear_model import LogisticRegression

In [13]:
from sklearn.model_selection import train_test_split

spam bot is the minimum level of spam to work with. I will start with this one.

In [14]:
df.columns

Index(['text', 'spam', 'spam_marketing', 'spam_hijack', 'spam_corporate',
       'spam_bot', 'spam_known', 'spam_own', 'Docusign', 'onespan', 'signnow',
       'adobe sign', 'listed_count', 'description', 'statuses_count',
       'followers_count', 'favourites_count', 'friends_count',
       'time_float_sin', 'time_float_cos', 'is_description_none',
       'text_without_accent', 'text_word_tokens', 'processed_text', 'stemmed',
       'ngrammed'],
      dtype='object')

In [15]:
df_model_training = df.iloc[:16000]
df_final_validation = df.iloc[16000:]

In [16]:
y = df_model_training['spam_bot']
X = df_model_training[['Docusign', 'onespan', 'signnow',
       'adobe sign', 'listed_count', 'statuses_count',
       'followers_count', 'favourites_count', 'friends_count',
       'time_float_sin', 'time_float_cos', 'is_description_none']]

In [17]:
X.head()

,Docusign,onespan,signnow,adobe sign,listed_count,statuses_count,followers_count,favourites_count,friends_count,time_float_sin,time_float_cos,is_description_none
0,False,False,False,True,1,6750,108,1,12,-0.049140,0.998792,False
1,False,False,False,True,14,5018,47,2,25,-0.501007,0.865443,False
2,False,False,False,True,0,29,60,10,52,-0.599548,0.800339,True
3,False,False,False,True,2,5332,66,0,182,-0.325843,0.945424,True
4,False,False,False,True,8,1337,225,19,348,-0.098162,0.995170,False


In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

In [19]:
logistic_model = LogisticRegression()

In [20]:
logistic_model.fit(X_train,y_train)

/home/daniel/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [21]:
y_predict = logistic_model.predict(X_test)

In [22]:
y_pred = logistic_model.predict_proba(X_test)

In [23]:
sum(y) / len(y)

0.727125

In [24]:
np.argmax(logistic_model.coef_)
X.columns[3]

'adobe sign'

In [25]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import log_loss
from sklearn.metrics import roc_auc_score

print(accuracy_score(y_test,y_predict))
print(confusion_matrix(y_test,y_predict))
print(log_loss(y_test,y_pred))
print(roc_auc_score(y_test,y_predict))

0.8905
[[ 684  399]
 [  39 2878]]
0.23594322765129233
0.8091045233928151


In [26]:
from sklearn.ensemble import RandomForestClassifier

In [27]:
rf_model = RandomForestClassifier()

In [28]:
rf_model.fit(X_train,y_train)

/home/daniel/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [29]:
y_predict = logistic_model.predict(X_test)

In [30]:
y_pred_rf = rf_model.predict_proba(X_test)

In [31]:
print(accuracy_score(y_test,y_predict))
print(confusion_matrix(y_test,y_predict))
print(log_loss(y_test,y_pred_rf))
print(roc_auc_score(y_test,y_predict))

0.8905
[[ 684  399]
 [  39 2878]]
0.12653338872384504
0.8091045233928151


In [32]:
from sklearn.ensemble import GradientBoostingClassifier

In [33]:
gb_model = GradientBoostingClassifier()

In [34]:
gb_model.fit(X_train,y_train)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              n_iter_no_change=None, presort='auto', random_state=None,
              subsample=1.0, tol=0.0001, validation_fraction=0.1,
              verbose=0, warm_start=False)

In [35]:
y_predict = gb_model.predict(X_test)

In [36]:
y_pred_gb = gb_model.predict_proba(X_test)

In [37]:
print(accuracy_score(y_test,y_predict))
print(confusion_matrix(y_test,y_predict))
print(log_loss(y_test,y_pred_gb))
print(roc_auc_score(y_test,y_predict))

0.988
[[1036   47]
 [   1 2916]]
0.05024673514275061
0.9781296067153069


This is understandable performance - every automation post is from Adobe Sign and there is a boolean column to see if the post is about Adobe Sign or not.

As the task was easy, good performance was understandable.

In [38]:
X.columns[np.argmax(logistic_model.coef_)]

'adobe sign'

In [39]:
#test_logistic_regressio(df_model_training, 'spam_bot')

In [40]:
def test_model(data,column,model):
    y = data[column]
    X = data[['Docusign', 'onespan', 'signnow','adobe sign',
                           'listed_count', 'statuses_count','followers_count',
                           'favourites_count', 'friends_count','time_float_sin',
                           'time_float_cos', 'is_description_none']]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)
    model = model()
    model.fit(X_train,y_train)
    y_predict = model.predict(X_test)
    y_pred = model.predict_proba(X_test)

    print("Accuracy score of this model is" + "\n")
    print(accuracy_score(y_test,y_predict))
    print("\n" + "Confusion Matrix of this model is" + "\n")
    print(confusion_matrix(y_test,y_predict))
    print("\n" + "Log-Loss score of this model is" + "\n")
    print(log_loss(y_test,y_pred))
    print("\n" + "AUC score of this model is" + "\n")
    print(roc_auc_score(y_test,y_predict))
    return None

In [41]:
from src.modeling import test_model

In [42]:
test_model(df_model_training, 'spam_bot',LogisticRegression)

Accuracy score of this model is

0.87475

Confusion Matrix of this model is

[[ 669  432]
 [  69 2830]]

Log-Loss score of this model is

0.3659455626339046

AUC score of this model is

0.7919140584980445


/home/daniel/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [43]:
test_model(df_model_training, 'spam_bot',RandomForestClassifier)

/home/daniel/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Accuracy score of this model is

0.98775

Confusion Matrix of this model is

[[1005   48]
 [   1 2946]]

Log-Loss score of this model is

0.17781685707911174

AUC score of this model is

0.9770383131428263


In [44]:
test_model(df_model_training, 'spam_bot',GradientBoostingClassifier)

Accuracy score of this model is

0.98775

Confusion Matrix of this model is

[[1048   45]
 [   4 2903]]

Log-Loss score of this model is

0.05209845446902725

AUC score of this model is

0.9787264611306714


In [45]:
test_model(df_model_training, 'spam_corporate',LogisticRegression)

/home/daniel/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Accuracy score of this model is

0.98525

Confusion Matrix of this model is

[[3803   12]
 [  47  138]]

Log-Loss score of this model is

0.09591626386602459

AUC score of this model is

0.8714002337855549


In [46]:
test_model(df_model_training, 'spam_corporate',RandomForestClassifier)

Accuracy score of this model is

0.99825

Confusion Matrix of this model is

[[3820    1]
 [   6  173]]

Log-Loss score of this model is

0.014399977624557563

AUC score of this model is

0.9831093676667755


/home/daniel/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [47]:
test_model(df_model_training, 'spam_corporate',GradientBoostingClassifier)

Accuracy score of this model is

0.9975

Confusion Matrix of this model is

[[3817    5]
 [   5  173]]

Log-Loss score of this model is

0.01204247818116352

AUC score of this model is

0.9853009483828103


In [48]:
test_model(df_model_training, 'spam_marketing',LogisticRegression)

/home/daniel/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Accuracy score of this model is

0.9545

Confusion Matrix of this model is

[[3816    4]
 [ 178    2]]

Log-Loss score of this model is

0.15108902423260984

AUC score of this model is

0.5050319953461315


In [86]:
test_model(df_model_training, 'spam_marketing',RandomForestClassifier)

Accuracy score of this model is

0.9625

Confusion Matrix of this model is

[[3788   27]
 [ 123   62]]

Log-Loss score of this model is

0.2991458300385499

AUC score of this model is

0.6640289043958769


/home/daniel/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [80]:
test_model(df_model_training, 'spam_marketing',GradientBoostingClassifier)

Accuracy score of this model is

0.95975

Confusion Matrix of this model is

[[3822   11]
 [ 150   17]]

Log-Loss score of this model is

0.10400742917192342

AUC score of this model is

0.5494632962095637


In [51]:
test_model(df_model_training, 'spam_hijack',LogisticRegression)

Accuracy score of this model is

0.917

Confusion Matrix of this model is

[[3116  205]
 [ 127  552]]

Log-Loss score of this model is

0.1951165384233035

AUC score of this model is

0.8756159202894598


/home/daniel/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [52]:
test_model(df_model_training, 'spam_hijack', RandomForestClassifier)

/home/daniel/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Accuracy score of this model is

0.93875

Confusion Matrix of this model is

[[3171  138]
 [ 107  584]]

Log-Loss score of this model is

0.3834630530909558

AUC score of this model is

0.9017237556302836


In [53]:
test_model(df_model_training, 'spam_hijack', GradientBoostingClassifier)

Accuracy score of this model is

0.94675

Confusion Matrix of this model is

[[3151  170]
 [  43  636]]

Log-Loss score of this model is

0.12827551851015398

AUC score of this model is

0.9427410875319685


In [54]:
df

,text,spam,spam_marketing,spam_hijack,spam_corporate,spam_bot,spam_known,spam_own,Docusign,onespan,...,favourites_count,friends_count,time_float_sin,time_float_cos,is_description_none,text_without_accent,text_word_tokens,processed_text,stemmed,ngrammed
0,just closed a deal in 27 hours using #AdobeSig...,0,0,0,0,1,0,0,False,False,...,1,12,-0.049140,0.998792,False,There's no substitute for experience! Call 609...,"[There, 's, no, substitute, for, experience, !...","[Theres, substitute, experience, Call, 6094028...","[there, substitut, experi, call, 6094028900, t...","[there, substitut, experi, call, 6094028900, t..."
1,just closed a deal in 2 days using #AdobeSign ...,0,0,0,0,1,0,0,False,False,...,2,25,-0.501007,0.865443,False,We provide an affordable way to have a wonderf...,"[We, provide, an, affordable, way, to, have, a...","[provide, affordable, way, wonderful, characte...","[provid, afford, way, wonder, charact, attend,...","[provid, afford, way, wonder, charact, attend,..."
2,just closed a deal in 2 hours using #AdobeSign...,0,0,0,0,1,0,0,False,False,...,10,52,-0.599548,0.800339,True,,[],[],[],[]
3,just closed a deal in 26 hours using #AdobeSig...,0,0,0,0,1,0,0,False,False,...,0,182,-0.325843,0.945424,True,,[],[],[],[]
4,just closed a deal in 6 days using #AdobeSign ...,0,0,0,0,1,0,0,False,False,...,19,348,-0.098162,0.995170,False,Carefully Crafted & Intentionally Inspired Eve...,"[Carefully, Crafted, &, Intentionally, Inspire...","[Carefully, Crafted, Intentionally, Inspired, ...","[care, craft, intent, inspir, event, plan, vin...","[care, craft, intent, inspir, event, plan, vin..."
5,just closed a deal in 35 minutes using #AdobeS...,0,0,0,0,1,0,0,False,False,...,0,182,-0.460587,0.887615,True,,[],[],[],[]
6,just closed a deal in 3 days using #AdobeSign ...,0,0,0,0,1,0,0,False,False,...,0,87,-0.305003,0.952351,False,SP Guard from Iconix uses enhancements to stan...,"[SP, Guard, from, Iconix, uses, enhancements, ...","[SP, Guard, Iconix, uses, enhancements, standa...","[SP, guard, iconix, use, enhanc, standardsbas,...","[SP, guard, iconix, use, enhanc, standardsbas,..."
7,just closed a deal in 58 minutes using #AdobeS...,0,0,0,0,1,0,0,False,False,...,4,17,-0.462587,0.886574,True,,[],[],[],[]
8,just closed a deal in 7 minutes using #AdobeSi...,0,0,0,0,1,0,0,False,False,...,14184,84426,-0.481626,0.876377,False,"Social Media Consultant, Real estate investor ...","[Social, Media, Consultant, ,, Real, estate, i...","[Social, Media, Consultant, Real, estate, inve...","[social, media, consult, real, estat, investor...","[social, media, consult, real, estat, investor..."
9,So apparently IU is catching on and wonвЂ™t ju...,0,0,0,0,0,0,0,False,False,...,15151,109,-0.673604,0.739092,False,BSU 17 - SIU 19 // Architecture & Interior Des...,"[BSU, 17, -, SIU, 19, //, Architecture, &, Int...","[BSU, 17, SIU, 19, Architecture, Interior, Des...","[bsu, 17, siu, 19, architectur, interior, desi...","[bsu, 17, siu, 19, architectur, interior, desi..."


In [55]:
df3 = df

In [56]:
df3['spam'][(df3['spam_bot']|df3['spam_corporate']==1)]=1

/home/daniel/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [57]:
df3

,text,spam,spam_marketing,spam_hijack,spam_corporate,spam_bot,spam_known,spam_own,Docusign,onespan,...,favourites_count,friends_count,time_float_sin,time_float_cos,is_description_none,text_without_accent,text_word_tokens,processed_text,stemmed,ngrammed
0,just closed a deal in 27 hours using #AdobeSig...,1,0,0,0,1,0,0,False,False,...,1,12,-0.049140,0.998792,False,There's no substitute for experience! Call 609...,"[There, 's, no, substitute, for, experience, !...","[Theres, substitute, experience, Call, 6094028...","[there, substitut, experi, call, 6094028900, t...","[there, substitut, experi, call, 6094028900, t..."
1,just closed a deal in 2 days using #AdobeSign ...,1,0,0,0,1,0,0,False,False,...,2,25,-0.501007,0.865443,False,We provide an affordable way to have a wonderf...,"[We, provide, an, affordable, way, to, have, a...","[provide, affordable, way, wonderful, characte...","[provid, afford, way, wonder, charact, attend,...","[provid, afford, way, wonder, charact, attend,..."
2,just closed a deal in 2 hours using #AdobeSign...,1,0,0,0,1,0,0,False,False,...,10,52,-0.599548,0.800339,True,,[],[],[],[]
3,just closed a deal in 26 hours using #AdobeSig...,1,0,0,0,1,0,0,False,False,...,0,182,-0.325843,0.945424,True,,[],[],[],[]
4,just closed a deal in 6 days using #AdobeSign ...,1,0,0,0,1,0,0,False,False,...,19,348,-0.098162,0.995170,False,Carefully Crafted & Intentionally Inspired Eve...,"[Carefully, Crafted, &, Intentionally, Inspire...","[Carefully, Crafted, Intentionally, Inspired, ...","[care, craft, intent, inspir, event, plan, vin...","[care, craft, intent, inspir, event, plan, vin..."
5,just closed a deal in 35 minutes using #AdobeS...,1,0,0,0,1,0,0,False,False,...,0,182,-0.460587,0.887615,True,,[],[],[],[]
6,just closed a deal in 3 days using #AdobeSign ...,1,0,0,0,1,0,0,False,False,...,0,87,-0.305003,0.952351,False,SP Guard from Iconix uses enhancements to stan...,"[SP, Guard, from, Iconix, uses, enhancements, ...","[SP, Guard, Iconix, uses, enhancements, standa...","[SP, guard, iconix, use, enhanc, standardsbas,...","[SP, guard, iconix, use, enhanc, standardsbas,..."
7,just closed a deal in 58 minutes using #AdobeS...,1,0,0,0,1,0,0,False,False,...,4,17,-0.462587,0.886574,True,,[],[],[],[]
8,just closed a deal in 7 minutes using #AdobeSi...,1,0,0,0,1,0,0,False,False,...,14184,84426,-0.481626,0.876377,False,"Social Media Consultant, Real estate investor ...","[Social, Media, Consultant, ,, Real, estate, i...","[Social, Media, Consultant, Real, estate, inve...","[social, media, consult, real, estat, investor...","[social, media, consult, real, estat, investor..."
9,So apparently IU is catching on and wonвЂ™t ju...,0,0,0,0,0,0,0,False,False,...,15151,109,-0.673604,0.739092,False,BSU 17 - SIU 19 // Architecture & Interior Des...,"[BSU, 17, -, SIU, 19, //, Architecture, &, Int...","[BSU, 17, SIU, 19, Architecture, Interior, Des...","[bsu, 17, siu, 19, architectur, interior, desi...","[bsu, 17, siu, 19, architectur, interior, desi..."


In [58]:
df_model_training = df3.iloc[:16000]
df_final_validation = df3.iloc[16000:]

In [59]:
test_model(df_model_training, 'spam', LogisticRegression)

/home/daniel/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Accuracy score of this model is

0.87825

Confusion Matrix of this model is

[[ 408  470]
 [  17 3105]]

Log-Loss score of this model is

0.27537290967101763

AUC score of this model is

0.729623627748698


In [60]:
test_model(df_model_training, 'spam', RandomForestClassifier)

/home/daniel/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Accuracy score of this model is

0.985

Confusion Matrix of this model is

[[ 832   50]
 [  10 3108]]

Log-Loss score of this model is

0.16102555698955603

AUC score of this model is

0.9700517367520025


In [61]:
test_model(df_model_training, 'spam', GradientBoostingClassifier)

Accuracy score of this model is

0.9835

Confusion Matrix of this model is

[[ 892   53]
 [  13 3042]]

Log-Loss score of this model is

0.062005006835920314

AUC score of this model is

0.9698300123832039


In [62]:
df3 = df
df3['spam'][(df3['spam_bot']|df3['spam_marketing']==1)]=1
df_model_training = df3.iloc[:16000]
df_final_validation = df3.iloc[16000:]

/home/daniel/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [63]:
test_model(df_model_training, 'spam', LogisticRegression)

/home/daniel/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Accuracy score of this model is

0.91775

Confusion Matrix of this model is

[[ 546  189]
 [ 140 3125]]

Log-Loss score of this model is

0.20968994722729398

AUC score of this model is

0.8499890614745133


In [64]:
test_model(df_model_training, 'spam', RandomForestClassifier)

Accuracy score of this model is

0.95525

Confusion Matrix of this model is

[[ 672   61]
 [ 118 3149]]

Log-Loss score of this model is

0.43665185948253565

AUC score of this model is

0.9403307956575971


/home/daniel/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [65]:
test_model(df_model_training, 'spam', GradientBoostingClassifier)

Accuracy score of this model is

0.95475

Confusion Matrix of this model is

[[ 654   45]
 [ 136 3165]]

Log-Loss score of this model is

0.12480740900850328

AUC score of this model is

0.9472113405613855


In [66]:
df3 = df
df3['spam'][(df3['spam_bot']|df3['spam_hijack']==1)]=1
df_model_training = df3.iloc[:16000]
df_final_validation = df3.iloc[16000:]

/home/daniel/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [67]:
test_model(df_model_training, 'spam', LogisticRegression)
test_model(df_model_training, 'spam', RandomForestClassifier)
test_model(df_model_training, 'spam', GradientBoostingClassifier)

/home/daniel/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/daniel/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/home/daniel/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Accuracy score of this model is

0.94875

Confusion Matrix of this model is

[[   0  199]
 [   6 3795]]

Log-Loss score of this model is

0.4162375867399547

AUC score of this model is

0.4992107340173639
Accuracy score of this model is

0.952

Confusion Matrix of this model is

[[  70  138]
 [  54 3738]]

Log-Loss score of this model is

0.3650122588591286

AUC score of this model is

0.6611489776046738
Accuracy score of this model is

0.95625

Confusion Matrix of this model is

[[  23  151]
 [  24 3802]]

Log-Loss score of this model is

0.1159351161476573

AUC score of this model is

0.5629555191040131


In [68]:
df3 = df
df3['spam'][(df3['spam_corporate']|df3['spam_marketing']==1)]=1
df_model_training = df3.iloc[:16000]
df_final_validation = df3.iloc[16000:]

/home/daniel/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [69]:
test_model(df_model_training, 'spam', LogisticRegression)
test_model(df_model_training, 'spam', RandomForestClassifier)
test_model(df_model_training, 'spam', GradientBoostingClassifier)

/home/daniel/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/daniel/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/home/daniel/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Accuracy score of this model is

0.946

Confusion Matrix of this model is

[[   4  204]
 [  12 3780]]

Log-Loss score of this model is

0.5292784703565733

AUC score of this model is

0.5080331061343719
Accuracy score of this model is

0.95675

Confusion Matrix of this model is

[[  60  130]
 [  43 3767]]

Log-Loss score of this model is

0.329841412258981

AUC score of this model is

0.6522516922226826
Accuracy score of this model is

0.95025

Confusion Matrix of this model is

[[  23  173]
 [  26 3778]]

Log-Loss score of this model is

0.1282403430077914

AUC score of this model is

0.5552560140775553


In [70]:
df3 = df
df3['spam'][(df3['spam_corporate']|df3['spam_hijack']==1)]=1
df_model_training = df3.iloc[:16000]
df_final_validation = df3.iloc[16000:]
test_model(df_model_training, 'spam', LogisticRegression)
test_model(df_model_training, 'spam', RandomForestClassifier)
test_model(df_model_training, 'spam', GradientBoostingClassifier)

/home/daniel/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/daniel/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/daniel/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/home/daniel/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Accuracy score of this model is

0.95625

Confusion Matrix of this model is

[[   4  169]
 [   6 3821]]

Log-Loss score of this model is

0.36863579087107085

AUC score of this model is

0.5107767898004897
Accuracy score of this model is

0.952

Confusion Matrix of this model is

[[  60  137]
 [  55 3748]]

Log-Loss score of this model is

0.3912764116855023

AUC score of this model is

0.6450531306435876
Accuracy score of this model is

0.95725

Confusion Matrix of this model is

[[  27  143]
 [  28 3802]]

Log-Loss score of this model is

0.11132298345341628

AUC score of this model is

0.5757564122254646


In [71]:
df3 = df
df3['spam'][(df3['spam_marketing']|df3['spam_hijack']==1)]=1
df_model_training = df3.iloc[:16000]
df_final_validation = df3.iloc[16000:]
test_model(df_model_training, 'spam', LogisticRegression)
test_model(df_model_training, 'spam', RandomForestClassifier)
test_model(df_model_training, 'spam', GradientBoostingClassifier)

/home/daniel/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/daniel/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/daniel/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/home/daniel/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Accuracy score of this model is

0.95525

Confusion Matrix of this model is

[[   1  179]
 [   0 3820]]

Log-Loss score of this model is

0.15398773553635467

AUC score of this model is

0.5027777777777778
Accuracy score of this model is

0.951

Confusion Matrix of this model is

[[  55  116]
 [  80 3749]]

Log-Loss score of this model is

0.3515128706343503

AUC score of this model is

0.650372121650867
Accuracy score of this model is

0.95425

Confusion Matrix of this model is

[[  31  157]
 [  26 3786]]

Log-Loss score of this model is

0.11436176657305377

AUC score of this model is

0.5790365251947935


In [72]:
df3 = df
df3['spam'][(df3['spam_bot']|df3['spam_corporate']|df3['spam_marketing']==1)]=1
df_model_training = df3.iloc[:16000]
df_final_validation = df3.iloc[16000:]
test_model(df_model_training, 'spam', LogisticRegression)
test_model(df_model_training, 'spam', RandomForestClassifier)
test_model(df_model_training, 'spam', GradientBoostingClassifier)

/home/daniel/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/daniel/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/daniel/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/home/daniel/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Accuracy score of this model is

0.9545

Confusion Matrix of this model is

[[   4  174]
 [   8 3814]]

Log-Loss score of this model is

0.5183520713270143

AUC score of this model is

0.510189382581036
Accuracy score of this model is

0.95075

Confusion Matrix of this model is

[[  69  126]
 [  71 3734]]

Log-Loss score of this model is

0.3474557055632175

AUC score of this model is

0.667593247750935
Accuracy score of this model is

0.95075

Confusion Matrix of this model is

[[  31  170]
 [  27 3772]]

Log-Loss score of this model is

0.1230531060987386

AUC score of this model is

0.5735608611326102


In [73]:
df3 = df
df3['spam'][(df3['spam_bot']|df3['spam_corporate']|df3['spam_hijack']==1)]=1
df_model_training = df3.iloc[:16000]
df_final_validation = df3.iloc[16000:]
test_model(df_model_training, 'spam', LogisticRegression)
test_model(df_model_training, 'spam', RandomForestClassifier)
test_model(df_model_training, 'spam', GradientBoostingClassifier)

/home/daniel/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/daniel/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/daniel/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/home/daniel/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Accuracy score of this model is

0.9495

Confusion Matrix of this model is

[[   1  199]
 [   3 3797]]

Log-Loss score of this model is

0.41312674875639366

AUC score of this model is

0.5021052631578947
Accuracy score of this model is

0.95075

Confusion Matrix of this model is

[[  55  128]
 [  69 3748]]

Log-Loss score of this model is

0.3171308583475798

AUC score of this model is

0.6412347121233595
Accuracy score of this model is

0.95175

Confusion Matrix of this model is

[[  31  166]
 [  27 3776]]

Log-Loss score of this model is

0.11952228807442382

AUC score of this model is

0.5751303739633818


In [74]:
df3 = df
df3['spam'][(df3['spam_bot']|df3['spam_hijack']|df3['spam_marketing']==1)]=1
df_model_training = df3.iloc[:16000]
df_final_validation = df3.iloc[16000:]
test_model(df_model_training, 'spam', LogisticRegression)
test_model(df_model_training, 'spam', RandomForestClassifier)
test_model(df_model_training, 'spam', GradientBoostingClassifier)

/home/daniel/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/daniel/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/daniel/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/home/daniel/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Accuracy score of this model is

0.9495

Confusion Matrix of this model is

[[   4  195]
 [   7 3794]]

Log-Loss score of this model is

0.4806471837539121

AUC score of this model is

0.5091294409432059
Accuracy score of this model is

0.9505

Confusion Matrix of this model is

[[  49  145]
 [  53 3753]]

Log-Loss score of this model is

0.3811978596436527

AUC score of this model is

0.6193259693051123
Accuracy score of this model is

0.956

Confusion Matrix of this model is

[[  29  162]
 [  14 3795]]

Log-Loss score of this model is

0.11313053679652527

AUC score of this model is

0.5740784776754971


In [75]:
df3 = df
df3['spam'][(df3['spam_corporate']|df3['spam_hijack']|df3['spam_marketing']==1)]=1
df_model_training = df3.iloc[:16000]
df_final_validation = df3.iloc[16000:]
test_model(df_model_training, 'spam', LogisticRegression)
test_model(df_model_training, 'spam', RandomForestClassifier)
test_model(df_model_training, 'spam', GradientBoostingClassifier)

/home/daniel/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/daniel/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Accuracy score of this model is

0.94975

Confusion Matrix of this model is

[[   0  201]
 [   0 3799]]

Log-Loss score of this model is

0.16955255235442637

AUC score of this model is

0.5
Accuracy score of this model is

0.95175

Confusion Matrix of this model is

[[  52  132]
 [  61 3755]]

Log-Loss score of this model is

0.2774985684229878

AUC score of this model is

0.6333116853522924


/home/daniel/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Accuracy score of this model is

0.952

Confusion Matrix of this model is

[[  31  168]
 [  24 3777]]

Log-Loss score of this model is

0.12151516474138364

AUC score of this model is

0.5747323833056364


In [76]:
df3 = df
df3['spam'][(df3['spam_bot']|df3['spam_hijack']|df3['spam_marketing']|df3['spam_corporate']==1)]=1
df_model_training = df3.iloc[:16000]
df_final_validation = df3.iloc[16000:]
test_model(df_model_training, 'spam', LogisticRegression)
test_model(df_model_training, 'spam', RandomForestClassifier)
test_model(df_model_training, 'spam', GradientBoostingClassifier)

/home/daniel/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/daniel/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/daniel/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/home/daniel/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Accuracy score of this model is

0.95275

Confusion Matrix of this model is

[[   6  172]
 [  17 3805]]

Log-Loss score of this model is

0.5204612691161518

AUC score of this model is

0.5146299660745889
Accuracy score of this model is

0.9545

Confusion Matrix of this model is

[[  46  124]
 [  58 3772]]

Log-Loss score of this model is

0.34110039410452286

AUC score of this model is

0.6277223160804792
Accuracy score of this model is

0.9585

Confusion Matrix of this model is

[[  35  149]
 [  17 3799]]

Log-Loss score of this model is

0.11923503941619688

AUC score of this model is

0.5928812323398049


Based on  testing so far, I found that numerical&boolean predictors are insufficient to predict spam_marketing and spam_hijack - which was expected.

However, surprisingly, spam_corporate and spam_bot were really doable (or more than that) for numerical&boolean predictors. It seems like these two criteria have pretty deep relationship with numerica/boolean predictors.

Now, I will have two different approaches:

1) categorized my TFIDF matrix to use it as predictors for one model (need to lower max_features for local computer)

2) ensemble of 2 Naive Bayes models & 1 classification model (RF/Logistic/GB)

Following the instruction of 